In [46]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras
import joblib

In [10]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [13]:
# Loads & precomputes all the data, takes about 2.5min
data = load_deals()

In [26]:
# Concatenating features
X = [np.array([h.feature_array() for h in d.hands]).flatten()
     for d in data]

# Winners in a particular strain played from N
y = [d.wnrs[strains.index('NT')]['N'] for d in data]
y_suit = [d.wnrs[strains.index('s')]['N'] for d in data]

In [ ]:
# PREVIOUS PERFORMANCE with old features, smaller dataset

# Features: Globally-defined X
#     Targets: Globally-defined y


#         M: Skl 1layNN #  test MAE: 0.76 - tr MAE: 0.72 - T: 256.4s # 40.2|86.2|97.8
#         M: HistGB     #  test MAE: 0.84 - tr MAE: 0.82 - T:   6.6s # 36.3|83.6|97.0
#         M: LinearRegr #  test MAE: 0.97 - tr MAE: 0.96 - T:   0.7s # 32.3|77.6|94.8


In [ ]:
#   10 iter = 1.5 min
#        M: Skl 1layNN #  test MAE: 0.80 - tr MAE: 0.79 - T: 59.6s # 38.0|84.9|97.5

#    200 iterations = 45 mins
#        M: Skl 1layNN #  test MAE: 0.73 - tr MAE: 0.70 - T: 2753.0s # 41.2|87.9|98.4

# 10 iter = 6 min
#        M: 4 layer 256 #  test MAE: 0.70 - tr MAE: 0.67 - T: 351.3s # 43.0|88.9|98.4
#        M: 7 layer 256 #  test MAE: 0.68 - tr MAE: 0.65 - T: 445.1s # 44.2|89.4|98.5
#           12 layer was same as 7, but worse training time

## results with mask instead
#        M: 7 layer 256 #  test MAE: 0.77 - tr MAE: 0.67 - T: 598.2s # 39.5|85.9|97.6


#    convergence = 44 iterations
#        M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8


In [ ]:
# sklearn 2h on NT

# Features: Globally-defined X
#     Targets: Globally-defined y

#         M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8
            
            
# sklearn 2h on s
# Features: Globally-defined X
#     Targets: SPADE WINNERS

#         M: Sklearn-NN #  test MAE: 0.45 - tr MAE: 0.37 - T: 5847.6s # 57.4|97.6|99.9

In [51]:
keras_m = train_eval_dds({'m': seq_model, 'desc': 'MLPNadamTan',
            'b_args': {'activation': 'relu', 'optimizer': keras.optimizers.Nadam(),
                       'layers_no': 6, 'dropout': False},
            'f_args': {'batch_size': 64, 'epochs': 3, 'verbose': 1}}, 
                         targets=[(y_suit, 'SPADES')], ret_model=True)  # 3min/epoch on CPU

Features: Globally-defined X
    Targets: SPADES

Epoch 1/3
337278/337278 [==============================] - 167s 496us/step - loss: 1.2828 - mean_absolute_error: 0.7956
Epoch 2/3
337278/337278 [==============================] - 131s 387us/step - loss: 0.6522 - mean_absolute_error: 0.6381
Epoch 3/3
337278/337278 [==============================] - 132s 390us/step - loss: 0.5859 - mean_absolute_error: 0.6049
        M: MLPNadamTan #  test MAE: 0.55 - tr MAE: 0.54 - T: 466.5s # 50.2|95.0|99.7


In [56]:
X[0]

array([18. ,  9. ,  6. ,  4. ,  3. ,  2.5,  0. ,  1. ,  0. ,  0. ,  1. ,
        2. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        4. ,  2. ,  1. ,  1. ,  0. ,  0. ,  1. ,  3. ,  1. ,  6. ,  1.5,
        1. ,  1. ,  0. ,  1. ,  0. ,  3. ,  1. ,  9. ,  3. ,  6.5,  3.5,
        7. ,  2. ,  1. ,  0. ,  1. ,  1. ,  0. ,  3. ,  1. ,  1. ,  1. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3. ,  1.5,  0. ,  0. ,
        1. ,  0. ,  0. ,  1. ,  1. ,  2. ,  2. ,  0. ,  0. ,  0. ,  0. ,
        1. ,  1. ,  0. ,  3. ,  2. , 10.5,  3. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  4. ,  4. ,  0. ,  0. ,  0. ,  1. ,
        0. ,  1. ,  0. ,  5. ,  4. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  4. ,  2.5,  0. ,  0. ,  1. ,  0. ,  0. ,  1. ,  1. , 10. ,
        5. ,  6. ,  4.5,  3. ,  3. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  8. ,  1. ,  1. ,  1. ,  1. ,  0. ,  1. ,  4. ,  1. ,  1. ,
        1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0

In [63]:
m[0].predict(X[0:100])

array([ 2.60583853,  3.64598458,  7.37382219,  4.12275891,  9.3202548 ,
        3.06433825,  5.0359051 ,  5.32037791,  8.51944752,  4.03769079,
        7.075908  ,  6.6500622 ,  8.36697908,  4.54569781,  9.95701212,
        2.17390773,  2.96764263,  3.43855461,  2.69976364,  9.75072989,
        6.49434525,  3.71691291,  1.31506868,  7.68612858,  5.88443624,
        8.89231508,  4.69594235,  8.44805049,  2.52921966,  9.59120139,
        5.47241374, 11.04818458,  3.05792188,  6.64585873,  8.68486304,
       11.84782044,  6.56868594,  6.40838005,  5.62709979,  5.30468277,
        9.69242343,  6.91052507,  6.09486283,  5.67396628,  7.53010235,
        7.62619352,  8.30978235,  5.14648903,  7.56317052,  0.30193716,
        7.84922776,  6.93870115,  8.47560096,  8.77646928,  1.11731591,
       11.72984953,  1.00801818,  6.78744291,  8.63452654,  3.19063148,
        2.3425592 ,  1.55546406,  6.8016907 ,  0.89937549,  7.16992583,
        4.56957224,  8.39171901,  5.3915187 ,  6.10810144,  5.80

In [65]:
keras_m[0].predict(X[0:1000])

array([[11.492817  ],
       [ 6.7625585 ],
       [ 8.784755  ],
       [ 7.5800686 ],
       [ 7.837754  ],
       [ 4.4164476 ],
       [ 4.4170012 ],
       [10.319663  ],
       [ 4.2586923 ],
       [ 2.9430182 ],
       [ 4.492983  ],
       [ 3.799145  ],
       [ 7.6136565 ],
       [ 8.107156  ],
       [11.1096525 ],
       [ 2.6377983 ],
       [ 3.0161362 ],
       [ 5.9583545 ],
       [ 2.255701  ],
       [ 9.800042  ],
       [ 2.364833  ],
       [ 3.2204022 ],
       [ 2.220639  ],
       [ 8.034862  ],
       [ 5.352472  ],
       [ 5.477219  ],
       [ 3.3708951 ],
       [ 8.373812  ],
       [ 2.0643811 ],
       [10.693585  ],
       [ 7.2494836 ],
       [ 7.5383067 ],
       [ 4.3108582 ],
       [ 4.1545763 ],
       [ 7.821796  ],
       [11.7563925 ],
       [ 4.7734203 ],
       [ 9.196158  ],
       [ 8.234374  ],
       [ 3.105874  ],
       [11.997874  ],
       [ 5.029301  ],
       [ 3.5588584 ],
       [ 3.822871  ],
       [ 4.088175  ],
       [ 5

In [52]:
myd = [0] * 10

In [68]:
for _ in range(10):
    myd[_] = Deal().solve()

In [70]:
for _ in range(10):
    myd[_] = myd[_].precompute()

In [ ]:
# Deal() = 26 ms
# Deal().solve() = 7s
# Deal().precompute() = 30ms

In [ ]:
# pl by N

In [111]:
mydeal = data[0]

ft = np.array([np.array([data[0].hands[(i + j) % 4].feature_array() for i in range(4)]).flatten()
      for j in range(4)])


In [115]:
a.apply(round_p)

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [119]:
v1[0]

11.492817

In [120]:
v1, v2, v3, v4 = a

SyntaxError: can't assign to function call (<ipython-input-120-3b2283c69461>, line 1)

In [113]:
a = keras_m[0].predict(ft)

In [131]:
ar = data[0].N.feature_array()

In [132]:
ar

array([18. ,  9. ,  6. ,  4. ,  3. ,  2.5,  0. ,  1. ,  0. ,  0. ,  1. ,
        2. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        4. ,  2. ,  1. ,  1. ,  0. ,  0. ,  1. ,  3. ,  1. ,  6. ,  1.5,
        1. ,  1. ,  0. ,  1. ,  0. ,  3. ,  1. ])

In [136]:
ar[31:], ar[4:13] = ar[4:13], ar[31:]

In [137]:
ar

array([18. ,  9. ,  6. ,  4. ,  3. ,  2.5,  0. ,  1. ,  0. ,  0. ,  1. ,
        2. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        4. ,  2. ,  1. ,  1. ,  0. ,  0. ,  1. ,  3. ,  1. ,  3. ,  2.5,
        0. ,  1. ,  0. ,  0. ,  1. ,  2. ,  1. ])

In [133]:
data[0].N.features

{'hcp': 18,
 'ctrl_p': 9,
 'losers': 6.0,
 'dist': 4.0,
 'c_len': 3,
 'c_losers': 2.5,
 'c_A': 0,
 'c_K': 1,
 'c_Q': 0,
 'c_J': 0,
 'c_T': 1,
 'c_hns': 2,
 'c_stop': 1,
 'd_len': 0,
 'd_losers': 0,
 'd_A': 0,
 'd_K': 0,
 'd_Q': 0,
 'd_J': 0,
 'd_T': 0,
 'd_hns': 0,
 'd_stop': 0,
 'h_len': 4,
 'h_losers': 2,
 'h_A': 1,
 'h_K': 1,
 'h_Q': 0,
 'h_J': 0,
 'h_T': 1,
 'h_hns': 3,
 'h_stop': 1,
 's_len': 6,
 's_losers': 1.5,
 's_A': 1,
 's_K': 1,
 's_Q': 0,
 's_J': 1,
 's_T': 0,
 's_hns': 3,
 's_stop': 1}

In [88]:
ft #pl by N, E, S, W

[array([18. ,  9. ,  6. ,  4. ,  3. ,  2.5,  0. ,  1. ,  0. ,  0. ,  1. ,
         2. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         4. ,  2. ,  1. ,  1. ,  0. ,  0. ,  1. ,  3. ,  1. ,  6. ,  1.5,
         1. ,  1. ,  0. ,  1. ,  0. ,  3. ,  1. ,  9. ,  3. ,  6.5,  3.5,
         7. ,  2. ,  1. ,  0. ,  1. ,  1. ,  0. ,  3. ,  1. ,  1. ,  1. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3. ,  1.5,  0. ,  0. ,
         1. ,  0. ,  0. ,  1. ,  1. ,  2. ,  2. ,  0. ,  0. ,  0. ,  0. ,
         1. ,  1. ,  0. ,  3. ,  2. , 10.5,  3. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  4. ,  4. ,  0. ,  0. ,  0. ,  1. ,
         0. ,  1. ,  0. ,  5. ,  4. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  4. ,  2.5,  0. ,  0. ,  1. ,  0. ,  0. ,  1. ,  1. , 10. ,
         5. ,  6. ,  4.5,  3. ,  3. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  8. ,  1. ,  1. ,  1. ,  1. ,  0. ,  1. ,  4. ,  1. ,  1. ,
         1. ,  0. ,  0. ,  0. ,  1. , 

In [82]:
data[0].E.pbn

'T3.Q62.5.AQJ8763'

In [79]:
data[0].hands[1].pbn

'T3.Q62.5.AQJ8763'

In [76]:
ft

array([18. ,  9. ,  6. ,  4. ,  3. ,  2.5,  0. ,  1. ,  0. ,  0. ,  1. ,
        2. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        4. ,  2. ,  1. ,  1. ,  0. ,  0. ,  1. ,  3. ,  1. ,  6. ,  1.5,
        1. ,  1. ,  0. ,  1. ,  0. ,  3. ,  1. ,  9. ,  3. ,  6.5,  3.5,
        7. ,  2. ,  1. ,  0. ,  1. ,  1. ,  0. ,  3. ,  1. ,  1. ,  1. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3. ,  1.5,  0. ,  0. ,
        1. ,  0. ,  0. ,  1. ,  1. ,  2. ,  2. ,  0. ,  0. ,  0. ,  0. ,
        1. ,  1. ,  0. ,  3. ,  2. , 10.5,  3. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  4. ,  4. ,  0. ,  0. ,  0. ,  1. ,
        0. ,  1. ,  0. ,  5. ,  4. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  4. ,  2.5,  0. ,  0. ,  1. ,  0. ,  0. ,  1. ,  1. , 10. ,
        5. ,  6. ,  4.5,  3. ,  3. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  8. ,  1. ,  1. ,  1. ,  1. ,  0. ,  1. ,  4. ,  1. ,  1. ,
        1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0

In [ ]:
#        M: MLPNadamTan #  test MAE: 0.55 - tr MAE: 0.55 - T: 431.2s # 49.8|95.0|99.8


In [93]:
joblib.dump(keras_m, 'models/ftarrays_to_dds_s_keras.pkl') 

['models/ftarrays_to_dds_s_keras.pkl']